请在Colab中打开：https://colab.research.google.com/github/Narwhalprime/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_cloud_natural_language_pipeline.ipynb

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点管道：云自然语言模型训练管道
<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_cloud_natural_language_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_cloud_natural_language_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/natural_language/cloud_natural_language_pipeline.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

概述
本笔记本展示如何使用[Google Cloud Pipeline Components SDK](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction)和本目录中的附加组件，在[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)中运行一个机器学习管道，用于训练一个TensorFlow文本分类模型。

在这个管道中，模型训练Docker镜像利用[TFHub](https://tfhub.dev/)模型来执行最先进的文本分类训练。这个镜像是预构建的且就绪可用，因此不需要额外的Docker设置。

### 目标

在本教程中，您将学习如何在Vertex AI管道中构建一个端到端的训练管道，该管道摄取一个数据集，在其上训练一个文本分类模型，并输出评估指标。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI管道
- Vertex AI数据集

执行的步骤包括：

- 定义Kubeflow管道组件
- 设置Kubeflow管道
- 在Vertex AI上运行管道

数据集

这个笔记本要求用户有两个从Vertex AI导出的数据集[托管数据集](https://cloud.google.com/vertex-ai/docs/training/using-managed-datasets)：一个包含训练和验证数据拆分，另一个包含用于评估的测试数据。请确保两个数据集之间没有共享数据（特别是评估数据不应该是训练或验证拆分的一部分）。要导出一个Vertex AI数据集，请按照以下公共文档操作：
* [准备数据](https://cloud.google.com/vertex-ai/docs/text-data/classification/prepare-data)
* [创建一个Vertex AI数据集](https://cloud.google.com/vertex-ai/docs/text-data/classification/create-dataset) 使用上述数据
* [导出数据集及其注释](https://cloud.google.com/vertex-ai/docs/datasets/export-metadata-annotations); 确保导出的结果位于您拥有的Google Cloud Storage（GCS）存储桶中。您可能需要手动将测试拆分数据分离到自己的文件中。

成本

本教程使用 Google Cloud 的收费组件：

* Vertex AI
* 云存储

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[云存储
定价](https://cloud.google.com/storage/pricing)，并使用[定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 设置

如果您正在使用Colab或Google Vertex AI Workbench Notebooks，则您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

***注意***：此笔记本已在以下环境中进行了测试：

* Python版本 = 3.8

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储 SDK
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的云存储指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简要的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)，并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 激活该环境，并在终端中运行 `pip3 install Jupyter` 来安装Jupyter。

5. 在终端中的命令行上运行 `jupyter notebook` 来启动Jupyter。

6. 在Jupyter Notebook仪表板中打开此笔记本。

### 安装附加包

运行以下命令来设置这个笔记本的包。请注意，本节中的最后一个代码片段将重新启动您的内核以正确加载安装内容，因此在从头开始初始化这个笔记本时，建议运行到该单元格，然后之后可以开始运行该单元格之后的单元格。

In [ ]:
# Install using pip3
!pip3 install -U tensorflow google-cloud-pipeline-components google-cloud-aiplatform kfp==1.8.16 "shapely<2" -q

In [ ]:
# Version check
# This has been tested with KFP 1.8.16
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，下面的步骤都是必要的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用用于支付计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

### 验证您的Google云帐号

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行验证您的帐号。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务帐号对项目的访问权限**部分，点击**角色**下拉列表。在筛选框中键入"Vertex AI"，并选择
   **Vertex AI管理员**。在筛选框中键入"Storage Object Admin"，并选择**存储对象管理员**。

5. 点击*创建*。将包含您密钥的JSON文件下载到您的本地环境。

6. 在下面的单元格中将您的服务帐号密钥路径作为
`GOOGLE_APPLICATION_CREDENTIALS` 变量输入并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置项目ID

在这里设置您的项目ID。如果您不知道项目ID，下面的代码将尝试从您的gcloud配置中确定。只有在笔记本可以看到您想要的项目时，请继续。

In [ ]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
print("Project ID:", PROJECT_ID)

In [ ]:
!gcloud config set project $PROJECT_ID

### 设置项目信息

在这里输入有关您的项目和数据集的信息。

In [ ]:
REGION = "us"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
TRAINING_DATA_LOCATION = "gs://your-training-data-location"  # @param {type:"string"}
TASK_TYPE = "CLASSIFICATION"  # @param ["CLASSIFICATION", "MULTILABEL_CLASSIFICATION"]

In [ ]:
# Since we are training a custom model, we need to specify the list of possible
# classes/labels.
# e.g, ["FirstClass", "SecondClass"]
# An additional class "[UNK]" will be added to the list indicating that none of
# the specified labels are a match.
CLASS_NAMES = [""]

# This is a list of GCS URIs; e.g., ["gs://your-bucket-name-here/your-input-file.jsonl"].
TEST_DATA_URIS = ["gs://your-bucket-name-here/your-input-file.jsonl"]

UUID

为了避免在项目中与其他资源发生名称冲突，您可以使用以下代码创建一个UUID，并将其附加到在此笔记本中创建的存储桶名称后面。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 创建云存储桶

**以下步骤是必需的，无论您的笔记本环境如何。**

当您初始化用于 Python 的 Vertex AI SDK 时，您会指定一个云存储暂存桶。暂存桶是您的数据集和模型资源相关数据在会话中保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括组织外部的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时，请运行以下单元格以创建您的云存储存储桶。

In [ ]:
!gsutil mb -l $REGION $BUCKET_URI

最后，通过检查存储桶的内容来验证对云存储桶的访问权限。

In [ ]:
!gsutil ls -al $BUCKET_URI

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

创建训练管道##

### 导入库

In [ ]:
from google_cloud_pipeline_components.aiplatform import ModelBatchPredictOp
from google_cloud_pipeline_components.experimental import natural_language
from google_cloud_pipeline_components.experimental.evaluation import (
    GetVertexModelOp, ModelEvaluationClassificationOp,
    TargetFieldDataRemoverOp)
from kfp import components
from kfp.v2 import compiler, dsl

### 定义常量

In [ ]:
# Worker pool specs
TRAINING_MACHINE_TYPE = "n1-highmem-8"
ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
ACCELERATOR_COUNT = 1
EVAL_MACHINE_TYPE = "n1-highmem-8"

## 定义组件

该流水线由以下组件组成：

- **train-tfhub-model** - 使用预构建的 Docker 镜像训练新的 Tensorflow 模型，使用 TFHub 层
- **upload-tensorflow-model-to-google-cloud-vertex-ai** - 将生成的模型上传到 Vertex AI 模型注册表
- **get-vertex-model** - 获取刚刚上传为管道中的一个工件的模型
- **convert-dataset-export-for-batch-predict** - 预处理组件，接收从 Vertex 数据集导出的测试数据集，并将其转换为可读取的简化兼容数据集，用于批量预测组件
- **target-field-data-remover** - 移除测试数据集中的目标字段（即标签），用于下游批处理预测组件
- **model-batch-predict** - 执行批量预测作业
- **model-evaluation-classification** - 从上述批量预测作业计算评估指标，并导出指标工件

In [ ]:
# Load upload TF model component
upload_tensorflow_model_to_vertex_op = components.load_component_from_url(
    "https://raw.githubusercontent.com/Ark-kun/pipeline_components/c6a8b67d1ada2cc17665c99ff6b410df588bee28/components/google-cloud/Vertex_AI/Models/Upload_Tensorflow_model/workaround_for_buggy_KFPv2_compiler/component.yaml"
)

### 定义管道

该管道执行以下步骤：
- 训练新的文本分类模型
- 将模型上传至 Vertex AI 模型注册表
- 对测试数据集进行预处理步骤导出：对于批量预测格式化数据，移除目标字段
- 对预处理后的测试数据进行批量预测
- 根据批量预测输出评估模型的表现

In [ ]:
@dsl.pipeline(name="text-classification-model")
def pipeline():
    train_task = natural_language.TrainTextClassificationOp()(
        project=PROJECT_ID,
        location=LOCATION,
        machine_type=TRAINING_MACHINE_TYPE,
        accelerator_type=ACCELERATOR_TYPE,
        accelerator_count=ACCELERATOR_COUNT,
        input_data_path=TRAINING_DATA_LOCATION,
        input_format="jsonl",
        natural_language_task_type=TASK_TYPE,
    )

    upload_task = upload_tensorflow_model_to_vertex_op(
        model=train_task.outputs["model_output"]
    )

    get_model_task = GetVertexModelOp(
        model_resource_name=upload_task.outputs["model_name"]
    )

    classification_type = (
        "multilabel" if TASK_TYPE == "MULTILABEL_CLASSIFICATION" else "multiclass"
    )

    convert_dataset_task = natural_language.ConvertDatasetExportForBatchPredictOp(
        file_paths=TEST_DATA_URIS, classification_type=classification_type
    )

    target_field_remover_task = TargetFieldDataRemoverOp(
        project=PROJECT_ID,
        location=LOCATION,
        root_dir=BUCKET_URI,
        gcs_source_uris=convert_dataset_task.outputs["output_files"],
        target_field_name="labels",
        instances_format="jsonl",
    )

    # Note: ModelBatchPredictOp doesn't support accelerators currently.
    batch_predict_task = ModelBatchPredictOp(
        project=PROJECT_ID,
        location=LOCATION,
        model=get_model_task.outputs["model"],
        job_display_name="nl-batch-predict-evaluation",
        gcs_source_uris=target_field_remover_task.outputs["gcs_output_directory"],
        instances_format="jsonl",
        predictions_format="jsonl",
        gcs_destination_output_uri_prefix=BUCKET_URI,
        machine_type=EVAL_MACHINE_TYPE,
    )

    # Note: Because we're running a custom training pipeline, the model source
    # is detected as Custom and thus it doesn't use AutoML NL's default settings
    # and fails if class_labels is excluded.
    ModelEvaluationClassificationOp(
        project=PROJECT_ID,
        location=LOCATION,
        root_dir=BUCKET_URI,
        class_labels=CLASS_NAMES + ["[UNK]"],
        predictions_gcs_source=batch_predict_task.outputs["gcs_output_directory"],
        predictions_format="jsonl",
        prediction_label_column="prediction.displayNames",
        prediction_score_column="prediction.confidences",
        ground_truth_gcs_source=convert_dataset_task.outputs["output_files"],
        ground_truth_format="jsonl",
        target_field_name="labels",
        classification_type=TASK_TYPE,
    )

### 编译管道

In [ ]:
compiler.Compiler().compile(pipeline, "nl_pipeline.json")

运行上面的代码行将在本地或Colab的目录中生成一个文件。

### 运行管道

这将向 Vertex Pipelines 发送一个创建管道作业请求。请注意，此任务是同步运行的，可能需要一段时间才能完成。

您可以随时通过单击生成的链接（在下面单元格的控制台输出中“查看管道作业”之后）来查看作业的进度。一旦管道完成，您可以检查从该管道产生的产物。

In [ ]:
job = aiplatform.PipelineJob(
    display_name="nl_pipeline",
    template_path="nl_pipeline.json",
    location=LOCATION,
    enable_caching=True,
    parameter_values={},
)

job.run()

一旦管道成功完成，请转到管道并检查结果指标的结果物。否则，请参考管道中失败的步骤，以确定任何错误的原因。

## 查看模型评估结果

要在管道执行后检查评估结果，请在由该管道创建的云存储存储桶中找到"model-evaluation-classification"子目录。您也可以运行以下命令直接输出度量文件的内容：

In [ ]:
import tensorflow as tf

EVAL_TASK_NAME = "model-evaluation-classification"
PROJECT_NUMBER = job.gca_resource.name.split("/")[1]
for _ in range(len(job.gca_resource.job_detail.task_details)):
    TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
    EVAL_METRICS = (
        BUCKET_URI
        + "/"
        + PROJECT_NUMBER
        + "/"
        + job.name
        + "/"
        + EVAL_TASK_NAME
        + "_"
        + str(TASK_ID)
        + "/executor_output.json"
    )
    if tf.io.gfile.exists(EVAL_METRICS):
        ! gsutil cat $EVAL_METRICS

清理资源

要清理此流水线使用的资源，请运行以下命令：

In [ ]:
# Delete GCS bucket.
!gsutil -m rm -r {BUCKET_URI}

下一步

如需另一种方法，请查看["ready-to-go"文本分类管道](https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/pipelines/google_cloud_pipeline_components_ready_to_go_text_classification_pipeline.ipynb)。该管道提供了模型逻辑以供进一步定制，如果需要的话，并添加了一个额外的管道步骤来部署模型以启用在线预测。